In [624]:
import numpy as np                    # Linear Algebra
import pandas as pd                   # Data processing 
import matplotlib.pyplot as plt       # Visualizations
import seaborn as sns                 # Visualizations
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,ShuffleSplit
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix,mean_squared_error 
from sklearn.metrics import classification_report,average_precision_score
import warnings                       # Hide warning messages
import networkx as nx
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.decomposition import IncrementalPCA
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")
%matplotlib inline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder

In [625]:
import tensorflow as tf
from  xgboost import XGBClassifier 

In [626]:
from pyfm import pylibfm

In [627]:
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, any_classifier,extra_trees;

In [670]:
df = pd.read_csv("../用户点击广告行为预测竞赛数据/my_train.csv").set_index('ID')
ID = pd.read_csv("../用户点击广告行为预测竞赛数据/my_train.csv")['ID']
X = df.drop(columns=['label'])
Y = pd.DataFrame(df['label'])
#X_test = pd.read_csv("../用户点击广告行为预测竞赛数据/test.csv")
#X['date'] = pd.to_datetime(X['date'])

In [671]:
neg_rate = Y['label'].value_counts()[0]/Y.shape[0]
neg_rate

0.8292666666666667

## Data preprocessing 

In [678]:
A = ['A1', 'A2', 'A3']
B = ['B1', 'B2', 'B3']
C = ['C1', 'C2', 'C3']
D = ['D1', 'D2']
E = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10',
       'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29']
E_no_cat =  ['E1', 'E2',  'E4', 'E5', 'E6', 'E7', 'E9', 'E10',
        'E12',  'E14', 'E15', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29']
E_cat = [ i for i in range(0,29)]
D_cat = [ i for i in range(29,36)]
F = ['F1', 'F2', 'F3', 'F4']
one_hot_columns = [
    'D1','D2','E4',
    'E8','E9','E10','E11','E13',
    'E15','E16','E17','E18','E19','E21',
    'E22','E25','E26','E28','E29'
]
wanted_columns = list(set(X.columns).difference(set(one_hot_columns)))

In [679]:
X_ = X.copy()
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[one_hot_columns])
h = enc.transform(df[one_hot_columns]).toarray()
h = pd.DataFrame(h)
h = h.join(ID).set_index('ID')
X_ = X_.drop(columns=one_hot_columns).join(h,on='ID');
mms = MinMaxScaler(feature_range=(1e-5, 1e+5))
for feat in wanted_columns:
    X_[feat] = mms.fit_transform(pd.DataFrame(X_[feat]))

In [640]:
# n_estimators=300
# max_depth=3
# min_samples_split=1
# learning_rate=0.1

# # 随机生成分类数据。
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
# # 将训练集切分为两部分，一部分用于训练GBDT模型，另一部分输入到训练好的GBDT模型生成GBDT特征，然后作为LR的特征。这样分成两部分是为了防止过拟合。
# X_train_GB, X_train_clf, Y_train_GB, Y_train_clf = train_test_split(X_train, Y_train, test_size=0.5)

# GBDT = GradientBoostingClassifier(n_estimators=300,learning_rate=learning_rate,min_samples_split=2,max_depth = 3)

# GBDT_enc = OneHotEncoder()
# # 调用LR分类模型。
# LR = LogisticRegression()

# '''使用X_train训练GBDT模型，后面用此模型构造特征'''
# GBDT.fit(X_train_GB, Y_train_GB)
# # fit one-hot编码器
# GBDT_enc.fit(GBDT.apply(X_train)[:, :, 0])

# #X_train_clf = GBDT_enc.transform(GBDT.apply(X_train_clf)[:, :, 0])
# #X_test_1 = GBDT_enc.transform(GBDT.apply(X_test)[:, :, 0])
# # X_test_LR = GBDT_enc.transform(GBDT.apply(X_test_LR)[:, :, 0])

In [682]:
X_train, X_test, Y_train, Y_test = train_test_split(X_, np.array(Y), test_size=0.3)

## Function Define

In [641]:
def print_score(esitmator,X_train, X_test, Y_train, Y_test,cv_):
    log_pred = esitmator.predict_proba(X_train)[:,1]
    print(f'AUC train : {roc_auc_score(Y_train, log_pred)}')
    cv1 = cross_val_score(log, X_train, Y_train, scoring = 'roc_auc',cv=cv_).mean()
    print(f'CV AUC train: {cv1}')
    
    log_pred_1 = esitmator.predict_proba(X_test)[:,1]
    print(f'AUC test : {roc_auc_score(Y_test, log_pred_1)}') 
    cv2 = cross_val_score(log, X_test, Y_test, scoring = 'roc_auc',cv=cv_).mean()
    print(f'CV AUC test : {roc_auc_score(Y_test, log_pred_1)}')    

In [642]:
class Classifier():
    def __init__(self,Y_train,Y_test):
        self.Y_train = Y_train;
        self.Y_test = Y_test;
        self.X_train = pd.DataFrame();
        self.X_test = pd.DataFrame();
    def update_data(self,X_train,X_test):
        self.X_train = X_train;
        self.X_test = X_test;
    def fit(self,estimator_,name):
        self.estimator = estimator_;
        self.estimator_name = name;
        self.estimator= self.estimator.fit(self.X_train,self.Y_train);
    def score(self):
        return cross_val_score(self.estimator, self.X_test, self.Y_test, scoring = 'roc_auc',cv=3).mean()
    def save(self):
        s = "../models/"+ estimator_name + ".m"
        joblib.dump(self.estimator, s);
    def print_score(self,cv_):
        pred  = self.estimator.predict_proba(self.X_train)[:,1]
        cv1 = cross_val_score(log, self.X_train, self.Y_train, scoring = 'roc_auc',cv=cv_).mean()
        pred_1 = self.estimator.predict_proba(X_test)[:,1]
        cv2 = cross_val_score(log, self.X_test, self.Y_test, scoring = 'roc_auc',cv=cv_).mean()
        
        print(f'AUC train : {roc_auc_score(self.Y_train, pred)}')
        print(f'CV AUC train: {cv1}')
        print(f'AUC test : {roc_auc_score(Y_test, pred_1)}') 
        print(f'CV AUC test : {cv2}')  
    def update_mode (self,estimator_,name):
        mode_dict = {}
        self.update_data(X_train,X_test);
        self.classify(estimator_,name)
        print(f"Mode1 : {self.score()}")
        mode_dict.update({'Mode1':self.model()})
        
        self.update_data(X_train[E_no_cat].join(X_train[E_cat]),\
                X_test[E_no_cat].join(X_test[E_cat]))
        self.classify(estimator_,name)
        print(f"Mode2 : {self.score()}")
        mode_dict.update({'Mode2':self.model()})
        
        self.update_data(X_train[E_no_cat].join(X_train[B]),\
                X_test[E_no_cat].join(X_test[B]))
        self.classify(estimator_,name)
        print(f"Mode3 : {self.score()}")
        mode_dict.update({'Mode3':self.model()})
        self.mode_dict = mode_dict
    def model(self):
        return self.estimator
    
    def result(self,estimator,X_train_,_Y_train_,X_test,Y_test):
        self.estimator = estimator
        estimator.fit(X_train_,Y_train_)
        pred  = estimator.predict_proba(X_test)[:,1]
        cv1 = cross_val_score(estimator, X_train_,Y_train_, scoring = 'roc_auc',cv=5).mean()
        cv2 = cross_val_score(estimator, X_test,Y_test, scoring = 'roc_auc',cv=5).mean()
        print(f'CV AUC train : {cv1}')
        print(f'CV AUC test : {cv2}')
        

In [564]:
clf = Classifier(Y_train,Y_test)

**Logistic Regression** 

In [434]:
log = LogisticRegression(C=1)

CV AUC test : 0.6849901637779973


In [562]:
X_train_LR.shape

(21000, 2318)

## SVM

In [ ]:
svm1 = GridSearchCV(svm.SVC(probability = True),\
                       {'kernel':['rbf'],\
                        'C':[0.0001],'gamma':[0.1]},cv=3,iid=False,\
                      return_train_score = True)
svm1.fit(X_train,Y_train);

In [ ]:
svm1.score(X_test,Y_test)

In [ ]:
svm_pred = svm1.predict(X_test)

In [ ]:
print(f"RMSE : {mean_squared_error(Y_test,svm_pred)}")

In [ ]:
print(classification_report(Y_test,svm_pred))

## Random Forest

In [450]:
rf = RandomForestClassifier()
clf.update_mode(rf,'rf')

Mode1 : 0.6128852892743364
Mode2 : 0.6242941144386934
Mode3 : 0.6276564929327288


In [558]:
rf = RandomForestClassifier()

In [567]:
rf.fit(X_train_LR,Y_train_LR)
pred  = rf.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(rf, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(rf, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

CV AUC train : 0.6363143946261729
CV AUC test : 0.6243059752035888


In [452]:
np.array(X_train.columns[np.argsort(rf.feature_importances_)])

array(['E15', 'E5', 'E2', 'E10', 'E4', 'E6', 'E9', 'C2', 'C3', 'B3',
       'E12', 'C1', 'B1', 'E14', 'A2', 'E17', 'E20', 'A3', 'E21', 'E19',
       'E23', 'E18', 'B2', 'A1', 'E22', 'E7', 'E1'], dtype=object)

In [ ]:
for i in range(X_train.shape[1]):
     score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2",
                              cv=ShuffleSplit(len(X), 3, .3))
     scores.append((round(np.mean(score), 3), names[i]))
print sorted(scores, reverse=True)

## DecisionTree 

In [456]:
dt2 = DecisionTreeClassifier(max_depth=3,max_features=len(X_.columns))

In [566]:
clf.result(dt2,X_train_LR,Y_train_LR,X_test_1,Y_test)

CV AUC train : 0.5022368661039807
CV AUC test : 0.623265254311066


In [569]:
dt2.fit(X_train_LR,Y_train_LR)
pred  = dt2.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(dt2, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(dt2, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

CV AUC train : 0.6241333454983283
CV AUC test : 0.588041603836213


In [461]:
# clf.update_mode(dt2,'');

In [243]:
np.array(X_train.columns[np.argsort(dt2.feature_importances_)])

array(['A1', 'E13', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20', 'E21', 'F3',
       'E22', 'E25', 'E26', 'E27', 'E28', 'E29', 'F5', 'F1', 'F2', 'E24',
       'E11', 'E12', 'E9', 'A2', 'B2', 'C1', 'C2', 'C3', 'D1', 'E10',
       'D2', 'F4', 'E2', 'E3', 'E5', 'E6', 'E7', 'E8', 'E1', 'A3', 'B3',
       'E14', 'E4', 'B1', 'E23'], dtype=object)

## Extra Tree 

In [112]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

In [139]:
new_xt = joblib.load("../models/extra_tree.m")

In [572]:
extratree = joblib.load("../models/extra_tree.m")
extratree

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [458]:
clf.update_mode(new_xt,'Extra_Tree')

Mode1 : 0.622411592743305
Mode2 : 0.6181002705582218
Mode3 : 0.6227649624787542


In [570]:
new_xt.fit(X_train_LR,Y_train_LR)
pred  = new_xt.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(new_xt, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(new_xt, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

KeyboardInterrupt: 

## KNN 

In [85]:
from sklearn.neighbors import KNeighborsClassifier
neigh =KNeighborsClassifier(n_neighbors=18)


In [ ]:
neigh.fit(X_train_LR,Y_train_LR)
pred  = neigh.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(neigh, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(neigh, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

In [573]:
neigh

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=18, p=2,
                     weights='uniform')

In [459]:
clf.update_mode(neigh,'KNN')

Mode1 : 0.6291495253529416
Mode2 : 0.6455139241547933
Mode3 : 0.6483281019704774


In [87]:
print_score(neigh,5)

AUC train : 0.7565103347382905
CV AUC train: 0.5344063153899461
AUC test : 0.6427708616788295
CV AUC test : 0.6427708616788295


In [88]:
joblib.dump(xg, "../models/knn.m");

## Bayes

In [187]:
from sklearn.preprocessing import LabelEncoder

In [462]:
from sklearn.naive_bayes import GaussianNB
gNB = GaussianNB();
clf.update_mode(gNB,'gNB')
gNB = clf.model_d

Mode1 : 0.5212953202070375
Mode2 : 0.5075292405398274
Mode3 : 0.5912357213187629


In [579]:
gNB.fit(X_train_LR.toarray(),Y_train_LR)
pred  = gNB.predict_proba(X_test_1.toarray())[:,1]
cv1 = cross_val_score(gNB, X_train_LR.toarray(),Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(gNB, X_test_1.toarray(),Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

CV AUC train : 0.510579285850448
CV AUC test : 0.5149893661112905


In [597]:
from sklearn.naive_bayes import MultinomialNB
#mNB = MultinomialNB(alpha = 0.1,fit_prior=False).fit(x_train,Y_train)
#clf.update_mode(mNB,'mNB')

In [598]:
mNB = MultinomialNB(alpha = 0.1,fit_prior=True)

In [605]:
mNB.fit(X_train_clf.toarray(),Y_train_LR)
pred  = mNB.predict_proba(X_test_1.toarray())[:,1]
cv1 = cross_val_score(mNB, X_train_clf.toarray(),Y_train_clf, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(mNB, X_test_1.toarray(),Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

CV AUC train : 0.6919270767637672
CV AUC test : 0.6872341950691349


In [607]:
roc_auc_score(Y_test,pred)

0.5537730496106515

In [464]:
from sklearn.naive_bayes import ComplementNB
cNB= ComplementNB().fit(x_train, Y_train)
clf.update_mode(cNB,'cNB')

Mode1 : 0.6064764052919749
Mode2 : 0.5802602906529509
Mode3 : 0.5990806419748672


In [ ]:
cNB.fit(X_train_LR,Y_train_LR)
pred  = cNB.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(cNB, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

In [191]:
from sklearn.naive_bayes import BernoulliNB
bNB = BernoulliNB()
bNB.fit(x_train, Y_train)
print_score(cNB,5)

AUC train : 0.5686438103839685
CV AUC train: 0.5344063153899461
AUC test : 0.5566481253774347
CV AUC test : 0.5566481253774347


In [ ]:
bNB.fit(X_train_LR,Y_train_LR)
pred  = bNB.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(bNB, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(bNB, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

## Bagging 

In [ ]:
bag = BaggingClassifier(base_estimator = {svm1,rf,log})

In [ ]:
plt.figure(figsize = (14, 6))
plt.plot(rf.feature_importances_)
plt.xticks(np.arange(X_train.shape[1]), X_train.columns.tolist(), rotation=90)
plt.show()

In [ ]:
a = list(zip(X_test, rf.feature_importances_))
a.sort(key=lambda tup:tup[1],reverse = True)

In [ ]:
sfm = SelectFromModel(rf,threshold = 0.01)
sfm.fit(X_train,Y_train);

In [ ]:
feat_labels = list(X_train.columns.values)
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
plt.figure(figsize=(14,6))
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

In [ ]:
X_important_train
rf_2 = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
rf_2 .fit(X_important_train,Y_train)

In [ ]:
print(classification_report(Y_test,rf_2.predict(X_important_test)))

## AdaBoost

In [482]:
Ada_3 = AdaBoostClassifier(n_estimators = 1000,learning_rate=1)

In [483]:
clf.update_mode(Ada_3,'Ada_3')

Mode1 : 0.6757219178931834
Mode2 : 0.6574455281519458
Mode3 : 0.6694477329326487


In [ ]:
Ada_3.fit(X_train_LR,Y_train_LR)
pred  = Ada_3.predict_proba(X_test_1)[:,1]
cv1 = cross_val_score(Ada_3, X_train_LR,Y_train_LR, scoring = 'roc_auc',cv=5).mean()
cv2 = cross_val_score(Ada_3, X_test_1,Y_test, scoring = 'roc_auc',cv=5).mean()
print(f'CV AUC train : {cv1}')
print(f'CV AUC test : {cv2}')

In [487]:
Ada_3 = clf.mode_dict['Mode1']
joblib.dump(Ada_3, '../models/Ada_3.m')

['../models/Ada_3.m']

In [ ]:
self.update_data(X_train,X_test);
self.classify(Ada_3,'Ada_3')

In [496]:
np.array(X_train.columns[np.argsort(Ada_3.feature_importances_)])

array(['E6', 'E10', 'E9', 'E15', 'E5', 'E4', 'E2', 'E14', 'E17', 'E12',
       'A3', 'A2', 'C3', 'E20', 'B1', 'E19', 'C1', 'C2', 'B3', 'E23',
       'E21', 'B2', 'E18', 'E7', 'A1', 'E1', 'E22'], dtype=object)

### 2

In [488]:
Ada_4 = AdaBoostClassifier(n_estimators = 1000,learning_rate=1)
Ada_4.fit(X_train.drop(columns=['C2','C3'],inplace= False),Y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
                   n_estimators=1000, random_state=None)

In [489]:
Ada_4_pred = Ada_4.predict_proba(X_test.drop(columns=['C2','C3'],inplace= False))[:,1]

In [492]:
print(roc_auc_score(Y_test,Ada_4_pred))
joblib.dump(Ada_3, '../models/Ada_4.m')

0.7010489319053267


['../models/Ada_4.m']

In [497]:
np.array(X_train.columns[np.argsort(Ada_4.feature_importances_)])

array(['E19', 14, 13, 'E23', 12, 11, 'E28', 'E29', 9, 'E20', 8, 'F3',
       'F4', 0, 1, 2, 3, 4, 5, 32, 6, 7, 'E17', 29, 28, 26, 24, 23, 22,
       15, 20, 21, 18, 17, 16, 'E12', 'E14', 'E15', 19, 10, 'E24', 25, 27,
       30, 31, 33, 'F2', 'E27', 'E9', 'E22', 'E26', 'C3', 'E21', 'E1',
       'E7', 'E6', 'F5', 'E2', 'E5', 'A1', 'F1', 'B3', 'B1', 'E4', 'E25',
       'E18', 'C2', 'B2', 'E10', 'C1', 'A3', 'A2'], dtype=object)

## RFE

In [228]:
from sklearn.feature_selection import RFE, RFECV
selector = RFE(estimator=Ada_3, n_features_to_select=10)
X_t = selector.fit_transform(X, Y)

In [229]:
X_train_t, X_test_t, Y_train_t, Y_test_t =\
train_test_split(X_t, Y, test_size=0.33, random_state=42)

In [232]:
Ada_3_t = AdaBoostClassifier(n_estimators = 1000,learning_rate=1)
Ada_3_t.fit(X_train_t,Y_train_t)
Ada_pred_3_t = Ada_3_t.predict_proba(X_test_t)
print(f'AUC : {roc_auc_score(Y_test, Ada_pred_3_t[:,1])}')

AUC : 0.6859697797600425


In [252]:
selector.ranking_

array([10,  1,  1,  5,  1,  4,  1,  1,  1, 17, 19,  1,  9, 25, 15, 13,  1,
        6, 32,  7, 14, 35, 36, 34,  1, 33, 31, 30, 24, 20,  2, 26, 27, 16,
       11, 22, 23,  1,  3, 18, 28, 29, 12,  8, 21])

In [287]:

rfecv = RFECV(estimator=svm.SVC(kernel='linear'), step=1, cv=StratifiedKFold(2),
              scoring='roc_auc')
rfecv.fit(X_train, Y_train['label'].replace({0:-1},inplace =False))

print("Optimal number of features : %d" % rfecv.n_features_)
print("Ranking of features : %s" % rfecv.ranking_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## XGBoost

In [44]:
from sklearn.externals import joblib

In [506]:
xt = XGBClassifier(max_depth =3, learning_rate = 0.1 ,n_estimators=300)
clf.update_mode(xt,'xt')
xt_dict = clf.mode_dict 

Mode1 : 0.6848385433694645
Mode2 : 0.6707116687555361
Mode3 : 0.6783584487802164


In [498]:
new_xt = joblib.load("../models/xgboost.m")

In [499]:
clf.update_mode(new_xt,'new_xt')


Mode1 : 0.6848385433694645
Mode2 : 0.6701092876835227
Mode3 : 0.6776553624085109


In [505]:
new_xt_dict = clf.mode_dict

In [512]:
joblib.dump(xt, "../models/xgboost2.m");

## LightGBM

In [522]:
from sklearn.ensemble import GradientBoostingClassifier
gbm = GridSearchCV(GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                        min_samples_leaf=20,subsample=0.8,random_state=2019),{'max_depth':[4],'n_estimators':[500]}, scoring='roc_auc',iid=False,cv=3);

clf.update_mode(gbm,'gbm')
gbm_dict = clf.mode_dict

Mode1 : 0.6791680616058938


KeyboardInterrupt: 

In [ ]:
max_depth = 4 ,n_estimators = 500

In [519]:
gbm_dict['Mode2'].best_params_

{'max_depth': 4, 'n_estimators': 500}

In [513]:
joblib.dump(gbm, "../models/gbm.m");

## Catboost


In [509]:
from catboost import CatBoostClassifier

In [511]:
cat = CatBoostClassifier(learning_rate=0.1,max_depth=5)
clf.update_mode(cat,'cat')
cat_dict = clf.mode_dict

0:	learn: 0.6391568	total: 26ms	remaining: 26s
1:	learn: 0.5987656	total: 43.6ms	remaining: 21.7s
2:	learn: 0.5635696	total: 53.9ms	remaining: 17.9s
3:	learn: 0.5372664	total: 63.9ms	remaining: 15.9s
4:	learn: 0.5178127	total: 75.2ms	remaining: 15s
5:	learn: 0.5007918	total: 84.4ms	remaining: 14s
6:	learn: 0.4874298	total: 95.5ms	remaining: 13.6s
7:	learn: 0.4763232	total: 122ms	remaining: 15.1s
8:	learn: 0.4679452	total: 144ms	remaining: 15.9s
9:	learn: 0.4607293	total: 169ms	remaining: 16.8s
10:	learn: 0.4553654	total: 195ms	remaining: 17.6s
11:	learn: 0.4510270	total: 227ms	remaining: 18.7s
12:	learn: 0.4469723	total: 252ms	remaining: 19.1s
13:	learn: 0.4441581	total: 276ms	remaining: 19.4s
14:	learn: 0.4415108	total: 301ms	remaining: 19.8s
15:	learn: 0.4397518	total: 322ms	remaining: 19.8s
16:	learn: 0.4384603	total: 347ms	remaining: 20s
17:	learn: 0.4367477	total: 368ms	remaining: 20.1s
18:	learn: 0.4351639	total: 387ms	remaining: 20s
19:	learn: 0.4337802	total: 407ms	remaining: 2

165:	learn: 0.4040798	total: 2.59s	remaining: 13s
166:	learn: 0.4037954	total: 2.61s	remaining: 13s
167:	learn: 0.4036374	total: 2.63s	remaining: 13s
168:	learn: 0.4034944	total: 2.65s	remaining: 13s
169:	learn: 0.4032944	total: 2.67s	remaining: 13s
170:	learn: 0.4031599	total: 2.68s	remaining: 13s
171:	learn: 0.4030163	total: 2.69s	remaining: 13s
172:	learn: 0.4027737	total: 2.7s	remaining: 12.9s
173:	learn: 0.4026745	total: 2.71s	remaining: 12.9s
174:	learn: 0.4025930	total: 2.72s	remaining: 12.8s
175:	learn: 0.4025326	total: 2.74s	remaining: 12.8s
176:	learn: 0.4024437	total: 2.76s	remaining: 12.8s
177:	learn: 0.4022228	total: 2.78s	remaining: 12.9s
178:	learn: 0.4020728	total: 2.81s	remaining: 12.9s
179:	learn: 0.4019784	total: 2.83s	remaining: 12.9s
180:	learn: 0.4018635	total: 2.85s	remaining: 12.9s
181:	learn: 0.4017596	total: 2.87s	remaining: 12.9s
182:	learn: 0.4016251	total: 2.89s	remaining: 12.9s
183:	learn: 0.4014846	total: 2.91s	remaining: 12.9s
184:	learn: 0.4013539	total

333:	learn: 0.3870213	total: 5.74s	remaining: 11.4s
334:	learn: 0.3869483	total: 5.76s	remaining: 11.4s
335:	learn: 0.3869191	total: 5.78s	remaining: 11.4s
336:	learn: 0.3867775	total: 5.79s	remaining: 11.4s
337:	learn: 0.3866988	total: 5.81s	remaining: 11.4s
338:	learn: 0.3866180	total: 5.82s	remaining: 11.3s
339:	learn: 0.3865179	total: 5.83s	remaining: 11.3s
340:	learn: 0.3864111	total: 5.83s	remaining: 11.3s
341:	learn: 0.3863361	total: 5.84s	remaining: 11.2s
342:	learn: 0.3861719	total: 5.85s	remaining: 11.2s
343:	learn: 0.3860794	total: 5.86s	remaining: 11.2s
344:	learn: 0.3859862	total: 5.87s	remaining: 11.1s
345:	learn: 0.3859152	total: 5.88s	remaining: 11.1s
346:	learn: 0.3858221	total: 5.89s	remaining: 11.1s
347:	learn: 0.3857942	total: 5.9s	remaining: 11.1s
348:	learn: 0.3857049	total: 5.91s	remaining: 11s
349:	learn: 0.3856245	total: 5.92s	remaining: 11s
350:	learn: 0.3855860	total: 5.93s	remaining: 11s
351:	learn: 0.3855389	total: 5.93s	remaining: 10.9s
352:	learn: 0.38545

498:	learn: 0.3753636	total: 9.31s	remaining: 9.35s
499:	learn: 0.3753321	total: 9.33s	remaining: 9.33s
500:	learn: 0.3753105	total: 9.35s	remaining: 9.31s
501:	learn: 0.3752221	total: 9.38s	remaining: 9.3s
502:	learn: 0.3752203	total: 9.39s	remaining: 9.28s
503:	learn: 0.3751276	total: 9.41s	remaining: 9.26s
504:	learn: 0.3750372	total: 9.43s	remaining: 9.25s
505:	learn: 0.3749146	total: 9.46s	remaining: 9.23s
506:	learn: 0.3748593	total: 9.48s	remaining: 9.22s
507:	learn: 0.3747591	total: 9.5s	remaining: 9.2s
508:	learn: 0.3746854	total: 9.52s	remaining: 9.19s
509:	learn: 0.3745844	total: 9.55s	remaining: 9.17s
510:	learn: 0.3745248	total: 9.57s	remaining: 9.16s
511:	learn: 0.3744041	total: 9.59s	remaining: 9.14s
512:	learn: 0.3743001	total: 9.62s	remaining: 9.13s
513:	learn: 0.3741680	total: 9.64s	remaining: 9.11s
514:	learn: 0.3741073	total: 9.66s	remaining: 9.1s
515:	learn: 0.3740054	total: 9.68s	remaining: 9.08s
516:	learn: 0.3739061	total: 9.71s	remaining: 9.07s
517:	learn: 0.37

663:	learn: 0.3642704	total: 13.1s	remaining: 6.64s
664:	learn: 0.3641967	total: 13.2s	remaining: 6.62s
665:	learn: 0.3641554	total: 13.2s	remaining: 6.61s
666:	learn: 0.3641158	total: 13.2s	remaining: 6.59s
667:	learn: 0.3640808	total: 13.2s	remaining: 6.57s
668:	learn: 0.3640384	total: 13.2s	remaining: 6.55s
669:	learn: 0.3640168	total: 13.3s	remaining: 6.54s
670:	learn: 0.3639743	total: 13.3s	remaining: 6.52s
671:	learn: 0.3639350	total: 13.3s	remaining: 6.5s
672:	learn: 0.3638736	total: 13.3s	remaining: 6.48s
673:	learn: 0.3638084	total: 13.4s	remaining: 6.46s
674:	learn: 0.3637629	total: 13.4s	remaining: 6.44s
675:	learn: 0.3637111	total: 13.4s	remaining: 6.42s
676:	learn: 0.3636591	total: 13.4s	remaining: 6.41s
677:	learn: 0.3635872	total: 13.5s	remaining: 6.39s
678:	learn: 0.3635253	total: 13.5s	remaining: 6.37s
679:	learn: 0.3634723	total: 13.6s	remaining: 6.38s
680:	learn: 0.3633946	total: 13.6s	remaining: 6.37s
681:	learn: 0.3633240	total: 13.6s	remaining: 6.36s
682:	learn: 0

...

289:	learn: 0.3917930	total: 2.95s	remaining: 7.23s
290:	learn: 0.3916300	total: 2.97s	remaining: 7.24s
291:	learn: 0.3915664	total: 2.99s	remaining: 7.25s
292:	learn: 0.3914938	total: 3s	remaining: 7.25s
293:	learn: 0.3914324	total: 3.01s	remaining: 7.24s
294:	learn: 0.3913957	total: 3.02s	remaining: 7.22s
295:	learn: 0.3912840	total: 3.03s	remaining: 7.22s
296:	learn: 0.3912013	total: 3.04s	remaining: 7.2s
297:	learn: 0.3911443	total: 3.05s	remaining: 7.19s
298:	learn: 0.3910998	total: 3.06s	remaining: 7.17s
299:	learn: 0.3910080	total: 3.07s	remaining: 7.16s
300:	learn: 0.3908963	total: 3.08s	remaining: 7.14s
301:	learn: 0.3908144	total: 3.08s	remaining: 7.13s
302:	learn: 0.3907914	total: 3.09s	remaining: 7.12s
303:	learn: 0.3907281	total: 3.1s	remaining: 7.11s
304:	learn: 0.3906074	total: 3.11s	remaining: 7.1s
305:	learn: 0.3905987	total: 3.12s	remaining: 7.09s
306:	learn: 0.3905417	total: 3.13s	remaining: 7.08s
307:	learn: 0.3904567	total: 3.14s	remaining: 7.06s
308:	learn: 0.3903

464:	learn: 0.3793476	total: 4.81s	remaining: 5.54s
465:	learn: 0.3792562	total: 4.83s	remaining: 5.54s
466:	learn: 0.3792414	total: 4.85s	remaining: 5.53s
467:	learn: 0.3791724	total: 4.87s	remaining: 5.53s
468:	learn: 0.3791681	total: 4.88s	remaining: 5.53s
469:	learn: 0.3791071	total: 4.89s	remaining: 5.51s
470:	learn: 0.3790175	total: 4.9s	remaining: 5.5s
471:	learn: 0.3789962	total: 4.91s	remaining: 5.49s
472:	learn: 0.3789367	total: 4.91s	remaining: 5.47s
473:	learn: 0.3788808	total: 4.92s	remaining: 5.46s
474:	learn: 0.3788757	total: 4.93s	remaining: 5.45s
475:	learn: 0.3788724	total: 4.94s	remaining: 5.44s
476:	learn: 0.3788613	total: 4.95s	remaining: 5.43s
477:	learn: 0.3787768	total: 4.96s	remaining: 5.42s
478:	learn: 0.3787041	total: 4.97s	remaining: 5.41s
479:	learn: 0.3786400	total: 4.98s	remaining: 5.39s
480:	learn: 0.3786043	total: 4.99s	remaining: 5.38s
481:	learn: 0.3784902	total: 5s	remaining: 5.37s
482:	learn: 0.3783962	total: 5.01s	remaining: 5.36s
483:	learn: 0.378

635:	learn: 0.3696689	total: 6.65s	remaining: 3.81s
636:	learn: 0.3696069	total: 6.67s	remaining: 3.8s
637:	learn: 0.3696031	total: 6.68s	remaining: 3.79s
638:	learn: 0.3695835	total: 6.7s	remaining: 3.79s
639:	learn: 0.3695327	total: 6.72s	remaining: 3.78s
640:	learn: 0.3694909	total: 6.73s	remaining: 3.77s
641:	learn: 0.3694724	total: 6.74s	remaining: 3.75s
642:	learn: 0.3694411	total: 6.74s	remaining: 3.74s
643:	learn: 0.3694133	total: 6.75s	remaining: 3.73s
644:	learn: 0.3693859	total: 6.76s	remaining: 3.72s
645:	learn: 0.3693451	total: 6.77s	remaining: 3.71s
646:	learn: 0.3693126	total: 6.78s	remaining: 3.7s
647:	learn: 0.3692914	total: 6.79s	remaining: 3.69s
648:	learn: 0.3692817	total: 6.8s	remaining: 3.68s
649:	learn: 0.3692788	total: 6.81s	remaining: 3.67s
650:	learn: 0.3692163	total: 6.82s	remaining: 3.65s
651:	learn: 0.3691037	total: 6.83s	remaining: 3.64s
652:	learn: 0.3690737	total: 6.84s	remaining: 3.63s
653:	learn: 0.3690489	total: 6.85s	remaining: 3.62s
654:	learn: 0.36

812:	learn: 0.3622282	total: 8.53s	remaining: 1.96s
813:	learn: 0.3621663	total: 8.55s	remaining: 1.95s
814:	learn: 0.3621314	total: 8.57s	remaining: 1.95s
815:	learn: 0.3620575	total: 8.59s	remaining: 1.94s
816:	learn: 0.3620321	total: 8.6s	remaining: 1.93s
817:	learn: 0.3620224	total: 8.62s	remaining: 1.92s
818:	learn: 0.3620115	total: 8.62s	remaining: 1.91s
819:	learn: 0.3619599	total: 8.63s	remaining: 1.89s
820:	learn: 0.3619479	total: 8.64s	remaining: 1.88s
821:	learn: 0.3619398	total: 8.65s	remaining: 1.87s
822:	learn: 0.3619015	total: 8.66s	remaining: 1.86s
823:	learn: 0.3618455	total: 8.66s	remaining: 1.85s
824:	learn: 0.3617957	total: 8.68s	remaining: 1.84s
825:	learn: 0.3617837	total: 8.68s	remaining: 1.83s
826:	learn: 0.3617372	total: 8.69s	remaining: 1.82s
827:	learn: 0.3617362	total: 8.7s	remaining: 1.81s
828:	learn: 0.3616985	total: 8.71s	remaining: 1.8s
829:	learn: 0.3616508	total: 8.72s	remaining: 1.78s
830:	learn: 0.3616465	total: 8.73s	remaining: 1.77s
831:	learn: 0.3

988:	learn: 0.3560790	total: 10.4s	remaining: 116ms
989:	learn: 0.3560617	total: 10.4s	remaining: 105ms
990:	learn: 0.3560330	total: 10.4s	remaining: 94.7ms
991:	learn: 0.3560158	total: 10.4s	remaining: 84.3ms
992:	learn: 0.3559697	total: 10.5s	remaining: 73.7ms
993:	learn: 0.3559436	total: 10.5s	remaining: 63.2ms
994:	learn: 0.3559035	total: 10.5s	remaining: 52.7ms
995:	learn: 0.3558999	total: 10.5s	remaining: 42.1ms
996:	learn: 0.3558935	total: 10.5s	remaining: 31.6ms
997:	learn: 0.3558725	total: 10.5s	remaining: 21ms
998:	learn: 0.3558704	total: 10.5s	remaining: 10.5ms
999:	learn: 0.3557994	total: 10.5s	remaining: 0us
Mode3 : 0.674547933632525


In [ ]:
cat@ = joblib.load("../models/cat.m")

In [ ]:
print_score(cat,5)

In [622]:
CatBoostClassifier?

In [514]:
joblib.dump(cat, "../models/cat.m");

##  MLP

In [690]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='sgd', activation='logistic',alpha=1e-5,hidden_layer_sizes=(50,5), random_state=1,max_iter=100,verbose=10,learning_rate_init=.1)

mlp.fit(X_train, Y_train)                         
print (mlp.n_layers_)
print (mlp.n_iter_)
print (mlp.loss_)
print (mlp.out_activation_)

Iteration 1, loss = 0.45942806
Iteration 2, loss = 0.45603377
Iteration 3, loss = 0.45585317
Iteration 4, loss = 0.45575534
Iteration 5, loss = 0.45594048
Iteration 6, loss = 0.45601438
Iteration 7, loss = 0.45597024
Iteration 8, loss = 0.45597329
Iteration 9, loss = 0.45589747
Iteration 10, loss = 0.45598861
Iteration 11, loss = 0.45576465
Iteration 12, loss = 0.45601831
Iteration 13, loss = 0.45586723
Iteration 14, loss = 0.45593151
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
4
14
0.45593151039454527
logistic


In [687]:
roc_auc_score(Y_train,mlp.predict_proba(X_train)[:,1])

0.5053249560991534